In [115]:
from final_structure.utils import _get_image
from final_structure.aws import get_table_markdown
from final_structure.gcs import make_request, format_text_from_annotations

In [116]:
image_path = '/Users/avinash/Desktop/Personal projects/ocr_to_layout-text/test_dataset/Demo Documents/Utility Bills/LGE5.pdf'

In [117]:
image = _get_image(image_path)

In [118]:
# (table_dict,dimension_dict)=get_table_markdown(image[0][0],image[0][1],image[0][2])

In [119]:
# gcs_response = make_request(image[0][0])

In [120]:
# gcs_extraction = format_text_from_annotations(gcs_response,dimension_dict,y_threshold=10,num_segments=70,x_close_threshold=100)

In [121]:
# tables = ""
# for idx,value in table_dict.items():
#     tables += f"#{idx}:\n\n {value}\n\n"

In [122]:
all_pages_extraction = []
for idx,image_idx in enumerate(image):
    gcs_response = make_request(image_idx[0])
    (table_dict,dimension_dict)=get_table_markdown(image_idx[0],image_idx[1],image_idx[2],idx)
    gcs_response = format_text_from_annotations(gcs_response,dimension_dict,y_threshold=10,num_segments=80,x_close_threshold=70)
    tables = ""
    for idx,value in table_dict.items():
        tables += f"#{idx}:\n\n {value}\n\n"
    final_Response = f"{gcs_response}\n\n{tables}"
    all_pages_extraction.append(final_Response)

In [123]:
# no_table = True
# if no_table:
#     dimension_dict = {}
# all_pages_extraction = []
# for idx,image_idx in enumerate(image):
#     gcs_response = make_request(image_idx[0])
#     gcs_response = format_text_from_annotations(gcs_response,dimension_dict,y_threshold=8,num_segments=80,x_close_threshold=50)
#     final_Response = gcs_response
#     all_pages_extraction.append(final_Response)

In [124]:
# final_Response = f"{gcs_extraction}\n\n{tables}"
# final_Response = f"{gcs_extraction}"

In [125]:
dir_paths = image_path.split("/")

In [126]:
filename=dir_paths[-1].split(".")[0]
filedir=dir_paths[-2]

In [127]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os

def strings_to_pdf(strings, width=1800, height=1500):  # Default size for A4
    # Register the font
    pdfmetrics.registerFont(TTFont('Monaco', 'MONACO.TTF'))
    os.makedirs(f"submission/pdf/{filedir}", exist_ok=True)
    
    # Create a new PDF with custom dimensions
    c = canvas.Canvas(f"submission/pdf/{filedir}/{filename}.pdf", pagesize=(width, height))
    
    for text in strings:
        c.setFont("Monaco", 12)  # Set the font and size
        
        # Split the text into lines
        lines = text.split('\n')
        
        # Draw each line on the PDF
        y = height - 50  # Start drawing 50 points from the top of the page
        for line in lines:
            c.drawString(50, y, line)  # Start drawing 50 points from the left of the page
            y -= 20  # Move down 20 points for the next line
        
        # End the current page and start a new one for the next string
        c.showPage()
    
    # Save the PDF after all strings have been processed
    c.save()
    
strings_to_pdf(all_pages_extraction)

In [128]:
def save_string_to_txt(string, file_path):
    with open(file_path, 'w') as file:
        file.write(string)
        
os.makedirs(f"submission/txt/{filedir}", exist_ok=True)
text = ""
for idx,page in enumerate(all_pages_extraction):
    text += page + "\n\n"
    tmp= "#"*10  + f"page{idx}" +  "#"*10 
    

file_path = f"submission/txt/{filedir}/{filename}.txt"
save_string_to_txt(text, file_path)
